In [49]:
import ee
import geemap
import ipywidgets as widgets
from ipyleaflet import WidgetControl
import os
import csv
import pandas as pd

In [67]:
#File with GBIF Data
CVS_FN=r"C:\Users\mike\Documents\OSU\GEOG581\Project\gbif\plantObservatins.csv"
PRES_LIST = ["Quercus agrifolia", "Heteromeles arbutifolia"]

#Path to the map
GBIF_MAP_PATH="users/karasofm/geog581_proj/treeDetection"

#Absense data is more prevelent, so be harder about uncertainty.
MAX_ABS_UNCERT = 5
MAX_PRES_UNCERT = 9

In [260]:
UNCERT_LBL="coordinateUncertaintyInMeters"
SPECIES_LBL="species"
LAT_LBL="decimalLatitude"
LONG_LBL="decimalLongitude"


csvReader = csv.DictReader(open(CVS_FN))
maxPresUncertainty = 9

max_lat = -90.0
max_long = -180.0
min_lat = 90.0
min_long = 180.0

for row in csvReader:
    try:
        uncert = int(row[UNCERT_LBL])
    except:
        continue
    
    lat=float(row[LAT_LBL])
    row[LAT_LBL]=lat
    long=float(row[LONG_LBL])
    row[LONG_LBL]=long
    
    if lat > max_lat:
        max_lat = lat
    if long > max_long:
        max_long = long
    
    if lat < min_lat:
        min_lat = lat
    if long < min_long:
        min_long = long


#Sort by nearest of each entry.  A KNN stick.
sortRef=[max_lat, max_long] 

def sortCoord(val):
    return ((sortRef[0]-val[LAT_LBL])**2 + (sortRef[1]-val[LONG_LBL])**2)

# for index in range(len(obsList)):
#     obsList[index:].sort(key=sortCoord)
#     sortRef=[obsList[index][LAT_LBL], obsList[index][LONG_LBL]] 

# for index in range(len(obsList)):
#     obsList[index]['spatialIdx'] = index

obsList.sort(key=sortCoord)
for index in range(len(obsList)):
     obsList[index]['spatialIdx'] = index

#Get ceneter
center_lat = min_lat + (max_lat-min_lat)/2 
center_long = min_long + (max_long-min_long)/2        

obsDf = pd.DataFrame(obsList) 

presDf = obsDf[obsDf[SPECIES_LBL].isin(PRES_LIST)]
absDf = obsDf[~obsDf[SPECIES_LBL].isin(PRES_LIST)]

absFc = geemap.pandas_to_ee(absDf, latitude=LAT_LBL, longitude=LONG_LBL)
presDf = geemap.pandas_to_ee(presDf, latitude=LAT_LBL, longitude=LONG_LBL)

In [261]:
Map = geemap.Map()
#naip_url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
#Map.add_wms_layer(url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True)

gbifAreaMap = ee.Image(GBIF_MAP_PATH)
Map.addLayer(gbifAreaMap);
Map.addLayer(absFc, {'color': 'red'}, 'Absense');
Map.addLayer(presDf, {'color': 'green'}, 'Presense');

Map.setCenter(center_long, center_lat, 10000)
Map

Map(center=[37.7627035, -122.44724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [262]:
obsDf.sort_values(by = 'spatialIdx')
currObs = []
currObs.append(obsDf[obsDf['spatialIdx']==0])oppppppppppp
allObs = []
al





lObs.append(obsDf)

fixedDf = currObs[0].drop(0)

def show_coord(obs):
    obsCoord=[float(obs[LONG_LBL].values[0]), float(obs[LAT_LBL].values[0])]
    Map.setCenter(obsCoord[0], obsCoord[1], 30)
    xy = ee.Geometry.Point(obsCoord)
    Map.addLayer(xy, {'color': 'yellow',
                     'pointRadius' : 1}, 'Select')

def next_obs():
    if not currObs[0].empty:
        nextSpIdx = currObs[0]['spatialIdx'].values[0] + 1
        currObs[0] = allObs[0][allObs[0]['spatialIdx']== nextSpIdx]
    
    return currObs[0]

def last_obs():
    nextSpIdx = currObs[0]['spatialIdx'].values[0] + 1
    nextObs = allObs[0][allObs[0]['spatialIdx']== nextSpIdx]
    
    if not nextObs.empty:
        currObs[0] = nextObs
        
    return currObs[0]

def get_obs():
    return currObs[0]



# Add an output widget to the map
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

keepBtn = widgets.Button(description="Keep")
keepBtn_control = WidgetControl(widget=keepBtn, position='topleft')
Map.add_control(keepBtn_control)

backBtn = widgets.Button(description="Back")
backBtn_control = WidgetControl(widget=backBtn, position='bottomright')
Map.add_control(backBtn_control)

skipBtn = widgets.Button(description="Skip")
skipBtn_control = WidgetControl(widget=skipBtn, position='topleft')
Map.add_control(skipBtn_control)


def on_skip_button_clicked(b):
    obs = next_obs()
    if(obs.empty):
        print("At End.")
    else:
        show_coord(obs)

def on_keep_button_clicked(b):
    obs = get_obs()
    fixedDf.append(obs)
    obs = next_obs()
    if(obs.empty):
        print("At End.")
    else:
        show_coord(obs)
        
def on_back_button_clicked(b):
    obs = last_obs()
    try:
        #If spatial index has been added to fixedDF, remove
        fixedDfRow = fixedDf[fixedDf['spatialIdx'] == 
                        obs['spatialIdx'].value[0]]
        fixedDf.drop(fixedDfRow.index[0])
    except IndexError:
        pass

    show_coord(obs)

skipBtn.on_click(on_skip_button_clicked)
backBtn.on_click(on_back_button_clicked)
keepBtn.on_click(on_keep_button_clicked)

# Capture user interaction with the map
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        # xy = ee.Geometry.Point(latlon[::-1])

        with output_widget:
            output_widget.clear_output()
            
            obs = get_obs()
            fixedDf.append(obs)

            obs = next_obs()
            if(obs.empty):
                print("At End.")
            else:
                show_coord(obs)
    
    Map.default_style = {'cursor': 'pointer'}

Map.on_interaction(handle_interaction)

show_coord(currObs[0])


In [223]:
fixedDf

""


In [95]:
import csv

with open(saveFile, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(fixedCoordList)